In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'gpu')

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset
from lib.ehr.inpatient_interface import Inpatients

In [3]:
# import logging
# logging.root.level = logging.DEBUG

In [4]:
# from lib.ehr.coding_scheme import MIMIC4Procedures, MIMIC4ProcedureGroups
# from lib.ehr.coding_scheme import MIMIC4Input, MIMIC4InputGroups

# cproc = MIMIC4Procedures()
# cproc_g = MIMIC4ProcedureGroups()
# cinp = MIMIC4Input()
# cinp_g = MIMIC4InputGroups()

In [5]:
# Assign the folder of the dataset to `DATA_FILE`.
import dask

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")

with U.modified_environ(DATA_DIR=DATA_DIR), dask.config.set(scheduler='processes'):
    m4inpatient_dataset = load_dataset('M4ICU')
   

In [6]:
splits = m4inpatient_dataset.random_splits([0.8, 0.9], random_seed=42, balanced='subjects')

In [7]:
preprocessing = m4inpatient_dataset.fit_preprocessing(splits[0])

In [8]:
m4inpatient_dataset.apply_preprocessing(preprocessing)

In [9]:
m4inpatients = Inpatients(m4inpatient_dataset)

In [10]:
# from concurrent.futures import ThreadPoolExecutor
# with dask.config.set(pool=ThreadPoolExecutor(12)):
with dask.config.set(scheduler='processes', num_workers=12):
    m4inpatients = m4inpatients.load_subjects(splits[0][:100], num_workers=12)

/home/asem/GP/ICENODE/notebooks/mimic_icu/../../lib/ehr/dataset.py:974: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  dob = anchor_date + anchor_age
                            dx_icd10->dx_icd9 Unrecognised t_codes
                            (169):
                            ['041.41', '041.42', '041.43', '041.49', '173.00', '173.01', '173.02', '173.09', '173.10', '173.11', '173.12', '173.19', '173.20', '173.21', '173.22', '173.29', '173.30', '173.31', '173.32', '173.39']...
                            dx_icd10->dx_icd9 Unrecognised s_codes
                            (49910):
                            ['E08.3211', 'E08.3212', 'E08.3213', 'E08.3219', 'E08.3291', 'E08.3292', 'E08.3293', 'E08.3299', 'E08.3311', 'E08.3312', 'E08.3313', 'E08.3319', 'E08.3391', 'E08.3392', 'E08.3393', 'E08.3399', 'E08.3411', 'E08.3412', 'E08.3413', 'E08.3419']...
                            dx_icd10->dx_icd9 Unrecognised t_codes
                           

In [11]:
m4inpatients.size_in_bytes() / 1024 ** 3

0.01845661737024784

In [12]:
# val_batch = m4inpatients.device_batch(splits[1])

In [13]:
# tst_batch = m4inpatients.device_batch(splits[2])

In [14]:
# val_batch.size_in_bytes() / 1024 ** 3, tst_batch.size_in_bytes() / 1024 ** 3

In [25]:
batch = m4inpatients.device_batch(splits[0][:32])

[autoreload of lib.ehr.inpatient_interface failed: Traceback (most recent call last):
  File "/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 305, in update_funct

Loading to device:   0%|          | 0/32 [00:00<?, ?subject/s]

In [16]:
batch.size_in_bytes() / 1024 ** 3

0.0056365299969911575

In [17]:
len(batch.subjects)

32

In [18]:
batch.n_admissions()

120

In [19]:
batch.n_segments()

7622

In [20]:
batch.n_obs_times()

5826

In [21]:
# import numpy as np
# import matplotlib.pyplot as plt

# a = m4inpatients_jax.obs_coocurrence_matrix
# a = np.array(a)
# plt.imshow(a, cmap='hot', interpolation='nearest')
# plt.show()

In [22]:
s = batch.subjects[splits[0][6]].admissions[0].interventions.input_
s

InpatientInput(
  index=i32[100],
  rate=f16[100],
  starttime=f32[100],
  endtime=f32[100],
  size=318
)

In [23]:
batch.interval_hours(splits[0][:10])

7167.583333333333

### التدريب على نموذج المعادلات التفاضلية الاعتيادية العصبية


In [24]:
from lib.ml.in_icenode import InICENODE, InICENODEDimensions
import jax.random as jrandom

/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/equinox/_ad.py:753: UserWarning: As of Equinox 0.10.7, `equinox.filter_custom_vjp.defvjp` is deprecated in favour of `.def_fwd` and `.def_bwd`. This new API supports symbolic zeros, which allow for more efficient autodifferentiation rules. In particular:
- the fwd and bwd functions take an extra `perturbed` argument, which     indicates which primals actually need a gradient. You can use this     to skip computing the gradient for any unperturbed value. (You can     also safely just ignore this if you wish.)
- `None` was previously passed to indicate a symbolic zero gradient for     all objects that weren't inexact arrays, but all inexact arrays     always had an array-valued gradient. Now, `None` may also be passed     to indicate that an inexact array has a symbolic zero gradient.
  warnings.warn(


NameError: name 'InICENode' is not defined

In [ ]:
dims = InICENODEDimensions(state_m=15, 
                state_dx_e=10,
                state_obs_e=25,
                input_e=10,
                proc_e=10,
                demo_e=5,
                int_e=15)
key = jrandom.PRNGKey(0)

m = InICENODE(dims=dims, 
              scheme=m4inpatient_dataset.scheme,
              key=key)

In [ ]:
res = m.batch_predict(batch, leave_pbar=True)